## Topics covered
1. Making socket url
2. Providing all required parameters
3. Create paper trading simulation platform
4. Using websocket and Binance to extract real time crypto data
5. Making and implementing the trading strategy
6. Running the code and final result
7. Analysing gain and loss

In [32]:
import websocket, numpy as np, json

In [33]:
symbol = "btcusdt"
interval = "1m"
socket = f"wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}"

amount = 1000
core_trade_amount = 0.8 * amount
trade_amount = 0.2 * amount
core_quantity = 0

# if core_to_trade == True, then we'll buy the entire 80% of the amount (core_trade_amount)
core_to_trade = True

portfolio = 0
investment, real_time_portfolio_value = [], []
closes, highs, lows, opens = [], [], [], []
account_bal = amount



In [34]:
# paper trading simulator
def buy(allocated_money, price):
    global portfolio, account_bal
    quantity = allocated_money / price
    portfolio += quantity
    money_end -= allocated_money

    if investment == []:
        investment.append(allocated_money)
    else:
        investment.append(allocated_money)
        investment[-1] += investment[-2]
    return quantity


def sell(allocated_money, price):
    global portfolio, account_bal
    quantity = allocated_money / price
    money_end -= quantity * price
    portfolio -= quantity

    investment.append(-allocated_money)
    investment[-1] += investment[-2]

In [35]:
def on_message(ws, message):
    global closes, highs, lows, opens, core_to_trade
    message = json.loads(message)
    # the actual data is in the 'k' key
    # other keys are metadata
    candle_stick = message["k"]
    candle_closed, close, open, high, low = candle_stick["x"], candle_stick["c"], candle_stick["o"], candle_stick["h"], candle_stick["l"]
    close, open, high, low = float(close), float(open), float(high), float(low)

    # this socket gives data every second, but we've give interval as 1m
    # so we'll only use the data when the candle is closed and that is represented by the x key
    # if x is true, then the candle is closed
    if candle_closed:
        closes.append(close)
        opens.append(open)
        highs.append(high)
        lows.append(low)
        last_price = closes[-1]
        
        
        print(f"Closes: {closes}")
        print("close", closes)
        print("open", opens)
        print("high", highs)
        print("low", lows)
        print("last_price", last_price)

        # this will only happen once in the beginning
        # we'll buy 80% of the amount, if core_to_trade is True
        # irrepective of the engulfing pattern
        if core_to_trade:
            buy(core_trade_amount, last_price)
            print(f"Core investment: We bought {core_trade_amount} worth of {symbol} at {last_price}")
            core_quantity = core_trade_amount / last_price
            core_to_trade = False
            

In [36]:
ws = websocket.WebSocketApp(socket, on_message=on_message)

ws.run_forever()

Closes: [19178.0]
close [19178.0]
open [19186.23]
high [19189.59]
low [19175.33]
last_price 19178.0
Closes: [19178.0, 19172.93]
close [19178.0, 19172.93]
open [19186.23, 19177.46]
high [19189.59, 19180.68]
low [19175.33, 19170.0]
last_price 19172.93
Closes: [19178.0, 19172.93, 19170.26]
close [19178.0, 19172.93, 19170.26]
open [19186.23, 19177.46, 19173.65]
high [19189.59, 19180.68, 19176.4]
low [19175.33, 19170.0, 19170.0]
last_price 19170.26


True